In [ ]:
// Need to set this true so that we can load the data without any pre defined schema
spark.conf.set("spark.sql.caseSensitive", "true")

Data Ingestion

In [1]:
// Loading the complete amazon review data set for Clothes Shoes and Jewelry
val clothesDF = spark.read.json("project/Clothing_Shoes_and_Jewelry.json")

In [2]:
// Loading the meta data for the above reviews data set
val metaDF = spark.read.json("project/meta_Clothing_Shoes_and_Jewelry.json")

In [3]:
clothesDF.printSchema()

In [4]:
metaDF.printSchema()

In [5]:
clothesDF.createOrReplaceTempView("amazon_data")

Viewing snippet of Data

In [6]:
// Showing a gist of Amazon reviews data
val ex = spark.sql("SELECT * FROM amazon_data")
ex.show()

In [7]:
metaDF.createOrReplaceTempView("amazon_metadata")


Viewing snippet of meta Data

In [8]:
// Showing a gist of Amazon reviews meta data
val ex1 = spark.sql("SELECT * FROM amazon_metadata")
ex1.show()

In [9]:
// Checking the data for "fit" column
val ex2 = spark.sql("SELECT distinct(rank) FROM amazon_metadata where rank is not NULL")
ex2.take(10)

In [10]:
// Data Profiling
// Total count of data rows 
val rows_total = clothesDF.count()
rows_total

In [11]:
// Data Profiling
// Total count of meta data rows 
val total_rows_metadata = metaDF.count()
total_rows_metadata

In [12]:
// List of columns originally present in the data
clothesDF.columns

In [13]:
// List of columns originally present in the meta data
metaDF.columns

Some part of Data profiling

In [14]:
// Data Profiling
// Number of nulls in each column of data
clothesDF.select(clothesDF.columns.map(colName => {
    count(when(col(colName).isNull, true)) as s"${colName}"
  }): _*)
  .show(12)

In [15]:
// Number of nulls in each column of meta-data
metaDF.select(metaDF.columns.map(colName => {
    count(when(col(colName).isNull, true)) as s"${colName}"
  }): _*)
  .show(19)

In [16]:
// Calculate total unique products in the data
val totalProducts = clothesDF.select(countDistinct("asin"))
totalProducts.show()

In [17]:
// Calculate total unique products in the meta data
val totalProductsMeta = metaDF.select(countDistinct("asin"))
totalProductsMeta.show()

In [18]:
// Count of reviews by verified users
clothesDF.filter(clothesDF("verified") === true).count()

In [19]:
// Count of reviews by non verified users
clothesDF.filter(clothesDF("verified") === false).count()

In [20]:
/**
 * Here we analyze each of the columns and decide whether we want to keep them or not using the data that we got in above paragraphs
 * 
 * asin : KEEP -> as this the unique product ID and we require it as a primary key in our database
 * image : REMOVE -> huge amount of nulls 
 * overall : KEEP -> tells us rating of each product given by a user, will help us in final search results
 * reviewText : REMOVE -> We are removing it as there is huge text which will hamper our results. We will use summary instead of this.
 * reviewTime : REMOVE -> we don't need it for our search results
 * reviewerID : REMOVE -> we don't need it for our search results
 * reviewerName : REMOVE -> we don't need it for our search results
 * style : REMOVE -> It has huge amount of null data
 * summary : KEEP -> This will help us in analyzing the reviews by past users and give search results to users
 * unixReviewTime : REMOVE -> we don't need it for our search results
 * verified : KEEP -> We see that there are more than 90% verified users, so we will remove the non-verified users.
 * vote : KEEP -> Even though there are lot of nulls, but this will help us in telling the intensity of the review.
 * 
 * Finally we will keep asin, overall, summary, verified and vote columns
 * 
**/

In [21]:
/**
 * Here we analyze each of the columns and decide whether we want to keep them or not using the data that we got in above paragraphs
 * 
 * also_buy : KEEP -> Will be a complementary suggestion to user
 * also_view : KEEP -> Will be a suggestion to user
 * asin : KEEP -> as this the unique product ID and we require it as a primary key in our database
 * brand : KEEP -> will be useful information to be given to the user
 * category : KEEP -> Will be an useful information.
 * date : REMOVE -> The given data is not related to date.
 * description : KEEP -> This will be useful in our search engine and also for the user to see.
 * details : REMOVE -> The data does not makes sense. It is showing as object but schema has type string.
 * feature : KEEP -> This will be useful information in search results and for the user display as well.
 * fit : REMOVE -> Data does not makes sense and there are lot of Nulls as well.
 * imageURL : REMOVE -> We are keeping the high resolution image URL, so don't need this. It will be redundant column.
 * imageURLHighRes : KEEP -> We can show the final image to the user in search result
 * main_cat : -> REMOVE -> This information is not useful for us as everything is related to Clothes, Shoes and Jewelry.
 * price : KEEP -> Required information to show it to the user.
 * rank : REMOVE -> The data doesn't make sense and is not relevant for search.
 * similar_item : REMOVE -> Since the data is object, we won't be able to get meaningful data out of it
 * tech1 : REMOVE -> Will not provide required information to us or the user
 * tech2 : REMOVE -> Will not provide required information to us or the user
 * title : KEEP -> will be useful information to be given to the user
 * 
 * Finally we will keep also_buy, also_view, asin, brand, category, description, feature, imageURLHighRes, price, similar_item, title
 * 
**/

Data Cleaning

In [22]:
// Dropping columns which are not required
val cleaned_data = clothesDF.drop( "image", "reviewText", "reviewTime", "reviewerID", "reviewerName", "style", "unixReviewTime")

In [23]:
// We are converting "vote" column from string to Integer

val cleaned_data1 = cleaned_data.withColumn("vote", cleaned_data("vote").cast("integer"))

In [24]:
cleaned_data1.show()

In [25]:
cleaned_data1.printSchema()

In [26]:
cleaned_data1.columns

In [27]:
// Dropping columns which are not required in meta data
val cleaned_metadata = metaDF.drop("date", "details", "fit", "imageURL", "main_cat", "similar_item", "rank", "tech1", "tech2")


In [28]:
cleaned_metadata.count()

In [29]:
// Data Profiling
// Removing redundant rows in the meta data
val unique_metadata = cleaned_metadata.distinct()

In [30]:
unique_metadata.count()

Data Cleaning continued

In [31]:
// Removing "$" sign from price column and converting it to string
// Some of rows of price have range like "12.29 - 17.89". So I am taking the maximum price for easier calculation.

import org.apache.spark.sql.functions._

val toDouble = udf[Double, String]( _.toDouble)

val unique_metadata1 = unique_metadata.withColumn(
    "price",
    expr("replace(price, '$', '')")
)

val unique_metadata2 = unique_metadata1.withColumn("price", split($"price", "\\-")(1)).select(
    "also_buy", "also_view", "asin", "brand", "category", "description", "feature", "imageURLHighRes", "title", "price"
)

val unique_metadata3 = unique_metadata2.withColumn(
    "price",
    expr("replace(price, '\\s', '')")
)

//   $"_tmp".getItem(1).as("max_price")

val unique_metadata4 = unique_metadata3.withColumn("price", unique_metadata3("price").cast("Double"))

// val unique_metadata4 = unique_metadata3.withColumn("price", toDouble(unique_metadata3("price"))).select("also_buy", "also_view", "asin", "brand", "category", "description", "feature", "imageURLHighRes", "price", "title")

In [32]:
unique_metadata3.columns

In [33]:
unique_metadata1.select("price").show()

In [34]:
unique_metadata4.select("price").show()

In [35]:
unique_metadata4.printSchema()

Joining meta data with full data

In [36]:
val joinedDf = cleaned_data1.join(unique_metadata4, Seq("asin"), "inner")

In [37]:
joinedDf.printSchema()

In [38]:
joinedDf.columns

In [39]:
joinedDf.select("asin", "overall", "summary", "verified", "vote", "also_buy", "also_view", "brand", "category", "description", "feature", "imageURLHighRes", "title", "price")

In [40]:
//"asin", "overall", "summary", "verified", "vote", "also_buy", "also_view", "brand", "category", "description", "feature", "imageURLHighRes", "title", "price"
joinedDf.select("asin").show()

In [41]:
// Data Profiling

Major Data Profiling

In [42]:
// Get all unique brand in the metadata

joinedDf.select("brand").distinct().show()

In [43]:
// Calculate minimum, maximum and average price of all the items in the metadata
joinedDf.agg(min("price"), max("price"), avg("price")).show()

In [44]:
// Calculate average rating per brand
val avg_rating = joinedDf.groupBy("brand").agg(mean("vote"))
avg_rating.show()

In [45]:
// Calculate average rating per product
val avg_rating_product = cleaned_data1.groupBy("asin").agg(mean("vote"))
avg_rating_product.show()

In [46]:
// Calculate total count of rating per product
val count_rating_product = cleaned_data1.groupBy("asin").count()
count_rating_product.show()

In [47]:
// See the minimum and maximum number of rating that a product got
count_rating_product.agg(min("count"), max("count")).show()